In [1]:
import os
import torch
from dataset.dataloader import DataloaderMode, create_dataloader
from model.model import Model
from model.model_arch import Net_arch
from utils.utils import calc_metric, get_multilingual_bert_tokenizer
from omegaconf import OmegaConf

model_checkpoint_name = "chkpt/vanila BERT_2022-12-14 21:28:42_1871.pt"
dataset_file_name = "dataset/meta/KorQuAD_v1.0_dev.json"
config_file_name = "config/job/train/default.yaml"

In [2]:
cfg = OmegaConf.load(config_file_name)
cfg.working_dir = os.getcwd()
cfg.job_logging_cfg = {'version': 1, 'formatters': {'simple': {'format': '%(message)s'}, 'detailed': {'format': '[%(asctime)s][%(name)s][%(levelname)s] - %(message)s'}}, 'handlers': {'console': {'class': 'logging.StreamHandler', 'formatter': 'detailed', 'stream': 'ext://sys.stdout'}, 'file': {'class': 'logging.FileHandler', 'formatter': 'detailed', 'filename': 'trainer.log'}}, 'root': {'level': 'INFO', 'handlers': ['console', 'file']}, 'disable_existing_loggers': False}
cfg.test.batch_size = 512
print(cfg)

{'name': 'NLP Final Project', 'working_dir': '/data/project/rw/andrew/snu/snu-nlp-2022', 'device': 'cuda', 'random_seed': 42, 'start_time': None, 'num_epoch': 8, 'data': {'train_file': 'dataset/meta/KorQuAD_v1.0_train.json', 'test_file': 'dataset/meta/KorQuAD_v1.0_dev.json', 'file_format': '*.file_extension', 'divide_dataset_per_gpu': True}, 'train': {'num_workers': 1, 'batch_size': 32, 'optimizer': {'mode': 'adam', 'adam': {'lr': 5e-05, 'betas': [0.9, 0.999]}}, 'scheduler': {'max_lr': '${train.optimizer.adam.lr}', 'min_lr': 1e-05}}, 'test': {'num_workers': 1, 'batch_size': 512}, 'model': None, 'loss': {'divisor': 2.0, 'custom_smoothing': False, 'order_const': 0.0}, 'dist': {'master_addr': 'localhost', 'master_port': '12355', 'mode': 'nccl', 'gpus': 0, 'timeout': 30}, 'log': {'use_tensorboard': False, 'use_wandb': True, 'wandb_init_conf': {'name': '${name}', 'entity': 'snu-nlp', 'project': 'KorBert'}, 'summary_interval': 1, 'chkpt_interval': 1, 'chkpt_dir': 'chkpt'}, 'load': {'wandb_lo

In [3]:
net_arch = Net_arch(cfg)
model = Model(cfg, net_arch, None)
model.load_network(torch.load(model_checkpoint_name, map_location=torch.device(cfg.device)))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
test_loader = create_dataloader(cfg, DataloaderMode.test, 0)

In [4]:
tokenizer = get_multilingual_bert_tokenizer()

In [9]:
def eval_model(cfg, model, test_loader, n_batches=-1):
    model.net.eval()
    total_test_loss = 0
    test_loop_len = 0
    sep_token = tokenizer.convert_tokens_to_ids("[SEP]")
    texts = dict()
    with torch.no_grad():
        for data in test_loader:
            if n_batches > -1 and test_loop_len >= n_batches:
                break
            print(f"Processing batch {test_loop_len}...")
            model_input, model_target, qa_id = data[:3], data[3:5], data[5]
            output = model.inference(model_input)

            # Calculate QA metrics
            batch_size = model_input[0].shape[0]
            num_words = output.shape[1]
            input_texts = model_input[0]

            start = torch.softmax(output[:, :, 0], dim=1)  # [batch_size, num_words]
            end = torch.softmax(output[:, :, 1], dim=1)  # [batch_size, num_words]
            s_rep = start.unsqueeze(dim=2).repeat(1, 1, num_words)  # [batch_size, num_words, num_words]
            e_rep = end.unsqueeze(dim=1).repeat(1, num_words, 1)  # [batch_size, num_words, num_words]
            scores = s_rep + e_rep
            scores = torch.triu(scores)  # [batch_size, num_words, num_words]

            sep_mask = input_texts == sep_token  # [batch_size, num_words]
            idx = torch.arange(sep_mask.shape[1], 0, -1)
            tmp = sep_mask * idx
            paragraph_starts = torch.argmax(tmp, 1) + 1
            paragraph_starts = (
                paragraph_starts.unsqueeze(dim=1).unsqueeze(dim=2).repeat(1, num_words, num_words)
            )  # [batch_size, num_words, num_words]
            indices1 = torch.arange(num_words).reshape(1, 1, num_words).repeat(batch_size, num_words, 1)
            scores[indices1 < paragraph_starts] = 0
            indices2 = torch.arange(num_words).reshape(1, num_words, 1).repeat(batch_size, 1, num_words)
            scores[indices2 < paragraph_starts] = 0

            scores = scores.reshape(batch_size, -1)
            max_idx = torch.argmax(scores, dim=1)
            
            pred_start = max_idx // num_words
            pred_end = max_idx % num_words

            for i in range(batch_size):
                pred_text = input_texts[i][pred_start[i] : pred_end[i] + 1].tolist()
                decoded_text = tokenizer.decode(pred_text)
                texts[qa_id[i]] = decoded_text

            test_loop_len += 1

        total_test_loss /= test_loop_len
        metric = calc_metric(cfg, texts)
    return texts, metric

texts, metric = eval_model(cfg, model, test_loader)
metric

Processing batch 0...
Processing batch 1...
Processing batch 2...
Processing batch 3...
Processing batch 4...
Processing batch 5...
Processing batch 6...
Processing batch 7...
Processing batch 8...
Processing batch 9...
Processing batch 10...


{'exact_match': 64.2362313820575, 'f1': 84.6992510938157}

In [14]:
flatten = " ".join(texts.values())
flatten.count("[UNK]")

0

In [8]:
for data in test_loader:
    model_input, model_target, qa_id = data[:3], data[3:5], data[5]
    decoded_text = tokenizer.decode(model_input[0][0].tolist())
    print(decoded_text)
    break

[CLS] 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? [SEP] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD